To obtain all the counterterms that emerge at each order in the perturbation expansion.

The different counterterm diagrams at each order will be of the form.

As the order increases the number of  

For example:
- 2nd order we would only have the counterterm 1-o-1.
- 3rd order we would have the counterterms 1-o-2 and 2-o-1.
- 4th order we would have: 1-o-1, 1-o-3, 3-o-1, 2-o-2.
- 5th order: 1-o-2, 2-o-1, 1-o-4, 4-o-1, 2-o-3, 3-o-2.
- 6th order: 1-o-1, 1-o-3, 3-o-1, 2-o-2, 1-o-5, 5-o-1, 2-o-4, 4-o-2, 3-o-3.
and so on.

So at each order we would have diagrams with the same appearance as the previous order of the same parity plus other diagrams which the sum of the number of incomming and outgoing particles equal the order considered.

Rememeber that in a perturbative expansion up till a certain order, we need to compute also the lower orders. Then we will need all the counterterms until that order.

For a certain theory, since the counterterms are the same for all interactions, we are interested in saving the counterterms in a file to be accessed in a latter calculation.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

#From Chatgpt
def trim_zeros_2D(array: np.ndarray, axis: int = 1) -> np.ndarray:
    """
    Remove any rows (axis=1) or columns (axis=0) that are entirely zero,
    even if they appear between non-zero rows/columns.
    
    Parameters
    ----------
    array : np.ndarray
        2D input array.
    axis : int, optional
        If 1 (default), drop zero-rows; if 0, drop zero-columns.
    
    Returns
    -------
    np.ndarray
        The trimmed array.
    """
    # mask[i] is True iff the i-th row/column has at least one non-zero
    mask = array.any(axis=axis)
    
    if axis:
        # drop rows where mask is False
        return array[mask, :]
    else:
        # drop columns where mask is False
        return array[:, mask]
    
#From chatgpt 
def trim_zeros_3D(array, axis=None):
    if axis is None:
        # Trim along all axes
        mask = ~(array == 0).all(axis=(1, 2))
        trimmed_array = array[mask]

        mask = ~(trimmed_array == 0).all(axis=(0, 2))
        trimmed_array = trimmed_array[:, mask]

        mask = ~(trimmed_array == 0).all(axis=(0, 1))
        trimmed_array = trimmed_array[:, :, mask]
    elif axis == 0:
        # Trim along axis 0
        mask = ~(array == 0).all(axis=(1, 2))
        trimmed_array = array[mask]
    elif axis == 1:
        # Trim along axis 1
        mask = ~(array == 0).all(axis=(0, 2))
        trimmed_array = array[:, mask]
    elif axis == 2:
        # Trim along axis 2
        mask = ~(array == 0).all(axis=(0, 1))
        trimmed_array = array[:, :, mask]
    else:
        raise ValueError("Invalid axis. Axis must be 0, 1, 2, or None.")
    
    return trimmed_array

#From Chatgpt
def find_equal_subarrays(array):
    """"
    Find the positions of duplicate subarrays in a 2D array.
    Args:
        array (np.array): 2D array to search for duplicates
    Returns:
        duplicate_positions (list): list of positions of duplicate
    """
    sorted_subarrays = [np.sort(subarray) for subarray in array]
    unique_subarrays, indices, counts = np.unique(sorted_subarrays, axis=0, return_index=True, return_counts=True)
    duplicate_positions = [np.where((sorted_subarrays == unique_subarrays[i]).all(axis=1))[0] for i in range(len(unique_subarrays)) if counts[i] > 1]
    return duplicate_positions

#From Chatgpt
def prune_points_and_reindex(points: np.ndarray,
                              paths: np.ndarray
                             ) -> tuple[np.ndarray, np.ndarray]:
    """
    Remove all [0,0] rows from `points`, then rebuild `paths` so that:
      - any path entry pointing to a removed point becomes 0,
      - all other entries are remapped down to a compact 1-based range.
    
    Parameters
    ----------
    points : np.ndarray, shape (N,2)
        Your (x,y) coordinates, with placeholder rows exactly equal to [0,0].
    paths : np.ndarray, shape (T,P,2), dtype=int
        Your 1-based index pairs, with [0,0] as placeholders.
    
    Returns
    -------
    new_points : np.ndarray, shape (M,2)
        The pruned points (no [0,0] rows).
    new_paths  : np.ndarray, shape (T,P,2)
        The updated paths, still 1-based with [0,0] placeholders.
    """
    # 1) Mask of rows to keep
    keep = ~(np.all(points == 0, axis=1))
    new_points = points[keep]
    
    # 2) Build old→new 1-based map
    #    new_idx[i] = new 1-based index of old row i, or 0 if dropped
    new_idx = np.zeros(points.shape[0], dtype=int)
    new_idx[keep] = np.arange(1, keep.sum()+1)
    
    # 3) Apply to paths
    #    For each entry u in paths: if u>0, replace with new_idx[u-1], else keep 0
    T, P, _ = paths.shape
    flat = paths.reshape(-1,2)
    # map both columns at once:
    mapped = np.zeros_like(flat)
    for col in (0,1):
        # grab the column, subtract 1 for 0-based indexing into new_idx
        o = flat[:,col]
        # for non-zero entries, look up new index; zeros stay zero
        mapped[:,col] = np.where(o>0, new_idx[o-1], 0)
    new_paths = mapped.reshape(T, P, 2)
    
    return new_points, new_paths

def represent_diagram (points, all_paths, index = False, directory = "", colors = ["tab:blue", "tab:red", "black"], line = ["solid", "solid", "photon"], count = 0):
    """
    Represent a diagram with points and paths.
    Args:
        points (np.array): array of points of dimension (n, 2)
        all_paths (list): list of paths to represent
        index (bool): whether to show the index of each point
        directory (str): directory to save the diagram
        colors (list): list of colors for each path
        line (list): list of line styles for each path
    """
    if (np.all(all_paths == 0)):
        return 
    
    points, all_paths = prune_points_and_reindex(points, all_paths)

    points = trim_zeros_2D(points)
    all_paths = trim_zeros_3D(all_paths, axis=1)
    
    fig=plt.figure(figsize=(5,3)) 
    ax=fig.add_subplot(111)
    ax.axis('off')
    j = 0

    # Note the that here the paths are more similar to the 1 particle case, meaning that is a 2D array
    for paths in all_paths:
        loops = find_equal_subarrays(paths)
        # Following the point made previously len(paths) indicate the number of connections instead of number of types of particles.
        for i in range(len(paths)):
            # In the case that the type of particle is a photon a spetial type of line is used to represent it.
            if (line[j] == "photon"):
                with mpl.rc_context({'path.sketch': (3, 15, 1)}):
                    if np.isin(i, loops):
                        middle_point = (points[paths[i, 0]-1] + points[paths[i, 1]-1]) / 2
                        circle = plt.Circle((middle_point[0], middle_point[1]), np.linalg.norm(points[paths[i, 0]-1]-middle_point), color=colors[j], fill=False)
                        ax.add_patch(circle)
                    elif paths[i, 0] == paths[i, 1] and paths[i ,0] != 0:
                        ax.scatter(points[paths[i, 0]-1, 0], points[paths[i, 0]-1, 1], color = colors[j])
                    else:
                        ax.plot([points[paths[i, 0]-1, 0], points[paths[i, 1]-1, 0]], [points[paths[i, 0]-1, 1], points[paths[i, 1]-1, 1]], color=colors[j])
            else:
                if np.isin(i, loops):
                    middle_point = (points[paths[i, 0]-1] + points[paths[i, 1]-1]) / 2
                    circle = plt.Circle((middle_point[0], middle_point[1]), np.linalg.norm(points[paths[i, 0]-1]-middle_point), color=colors[j], fill=False, linestyle=line[j])
                    ax.add_patch(circle)
                elif paths[i, 0] == paths[i, 1] and paths[i ,0] != 0:
                    ax.scatter(points[paths[i, 0]-1, 0], points[paths[i, 0]-1, 1], color = colors[j], s = 50, zorder = 10)
                else:
                    ax.plot([points[paths[i, 0]-1, 0], points[paths[i, 1]-1, 0]], [points[paths[i, 0]-1, 1], points[paths[i, 1]-1, 1]], color=colors[j], linestyle=line[j])
        j+=1
    ax.axis('equal')
    if index:
        for i in range(len(points)):
            ax.text(points[i, 0], points[i, 1], str(i+1), fontsize=12, color="black", ha="right", va="top")
    if count !=0:
        ax.text(0.5, 0.5, f"N = {count}", fontsize=12, color="black", ha="center", va="center")
    if directory != "":
        plt.savefig(directory, bbox_inches='tight')
        plt.close() #Added to not show in the notebook 

In [7]:
def return_counterterm_diagrams (order):
    """
    
    """
    #initialize the list of points and paths where the diagrams will be stored, and returned at the end.
    all_points = []
    all_paths = []
    #iterate through the order in perturbation (variable i)
    for i in range(2, order + 1):
        #initialize the list of points and paths for this order
        points_orders = []
        paths_orders = []
        #at each order, compute the counterterm diagrams
        #where j is the number of outgoing particles

        if (i > 3):
            if (i%2 == 0):
                j = i-4
                for l in range(len(all_points[j])):
                    points_orders.append(all_points[j][l])
                for l in range(len(all_paths[j])):
                    paths_orders.append(all_paths[j][l])

            if (i%2 == 1):
                j = i-4
                for l in range(len(all_points[j])):
                    points_orders.append(all_points[j][l])
                for l in range(len(all_paths[j])):
                    paths_orders.append(all_paths[j][l])

        for j in range(1, i):
            """
            Starting with the points for the diagrams.
            """
            #initialize a dummy array variable for each diagram
            points = []
            #k is the number of incoming particles
            k = i - j
            #create the outgoing points for the diagram
            for l in range(1, j+1):
                points.append([1, l])
            #create the point where the counterterm is located
            points.append([2, 1])
            #create the incoming points for the diagram
            for l in range(1, k+1):
                points.append([3, l])
            #append the points for this diagram
            points_orders.append(points)

            """
            Following a similar procedure for the paths.
            """
            #initialize a dummy array variable for each path
            paths = []
            for l in range(1, j+1):
                #create the outgoing paths for the diagram
                paths.append([l, j+1])
            paths.append([j+1, j+1])
            for l in range(1, k+1):
                #create the incoming paths for the diagram
                paths.append([j+1, j+l+1])
            #append the paths for this diagram
            paths_orders.append(paths)
            
        #append the points for this order
        all_points.append(points_orders)

        #append the paths for this order
        all_paths.append(paths_orders)
    
    return all_points, all_paths

n=7

counter_points, counter_paths = return_counterterm_diagrams(n)

for i in range(len(counter_points)):
    for j in range(len(counter_points[i])):
        represent_diagram(np.array(counter_points[i][j]), np.array([counter_paths[i][j],np.zeros_like(counter_paths[i][j])]), 
                          index = True, colors=["black", "black"], directory=f"counterterms_test/order{i+2}/diagram{j+1}.png")

In [24]:
import numpy as np

def combine_paths(*paths):
    """ 
    Combine multiple paths into a single array with the same length by padding with zeros.
    Args:
        paths (list): list of paths to combine
    Returns:
        final_path (np.array): combined path of dimension (len(paths), max_len, 2)
    """
    max_len = max([len(path) for path in paths])
    final_path = np.zeros((len(paths), max_len, 2), dtype=int)
    for i in range(len(paths)):
        if len(paths[i]) < max_len:
            final_path[i] = np.append(paths[i], np.zeros((max_len - len(paths[i]), 2), dtype=int), axis=0)
        else:
            final_path[i] = paths[i]
    return final_path

# First order
points_1st_1 = np.array([[0, 1], [0, 3], [1, 2], [2, 2]])
paths_1st_1g = np.array([[1, 3], [2, 3], [3, 4]]) 
paths_1st_1i = np.array([[0, 0]])

paths_1st_1 = combine_paths(paths_1st_1g, paths_1st_1i)

points_1st_2 = np.array([[0, 2], [1, 2], [2, 1], [2, 3]])
paths_1st_2g = np.array([[1, 2], [2, 3], [2, 4]]) 
paths_1st_2i = np.array([[0, 0]])

paths_1st_2 = combine_paths(paths_1st_2g, paths_1st_2i)

can_points_1st = np.empty((2, max(len(points_1st_1), len(points_1st_2)), 2))
can_points_1st[0] = points_1st_1
can_points_1st[1] = points_1st_2

can_paths_1st = np.empty((2, max(len(paths_1st_1), len(paths_1st_2)), max(len(paths_1st_1[0]), len(paths_1st_2[0])), 2), dtype=int)
can_paths_1st[0] = paths_1st_1
can_paths_1st[1] = paths_1st_2

can_number_1st = np.array([[1], [1], [1], [1]])

#Second order
points_2nd_1_1 = np.array([[0, 1], [0, 3], [0, 5], [1, 4], [2, 3], [4, 1]])
paths_2nd_1_1g = np.array([[1, 5], [2, 4], [3, 4], [5, 6]])
paths_2nd_1_1i = np.array([[4, 5]])
paths_2nd_1_1 = combine_paths(paths_2nd_1_1g, paths_2nd_1_1i)

points_2nd_1_2 = np.array([[0, 1], [0, 3], [0, 5], [1, 2], [2, 3], [4, 1]])
paths_2nd_1_2g = np.array([[1, 4], [2, 4], [3, 5], [5, 6]])
paths_2nd_1_2i = np.array([[4, 5]])
paths_2nd_1_2 = combine_paths(paths_2nd_1_2g, paths_2nd_1_2i)

points_2nd_1_3 = np.array([[0, 1], [0, 3], [0, 5], [1, 4], [2, 3], [4, 1]])
paths_2nd_1_3g = np.array([[1, 4], [2, 5], [3, 4], [5, 6]])
paths_2nd_1_3i = np.array([[4, 5]])
paths_2nd_1_3 = combine_paths(paths_2nd_1_3g, paths_2nd_1_3i)

points_2nd_2_1 = np.array([[0, 1], [0, 3], [1, 2], [2, 2], [3, 1], [3, 3]])
paths_2nd_2_1g = np.array([[1, 3], [2, 3], [4, 5], [4, 6]])
paths_2nd_2_1i = np.array([[3, 4]])
paths_2nd_2_1 = combine_paths(paths_2nd_2_1g, paths_2nd_2_1i)

points_2nd_2_2 = np.array([[0, 1], [0, 3], [1, 1], [2, 3], [3, 1], [3, 3]])
paths_2nd_2_2g = np.array([[1, 3], [2, 4], [3, 5], [4, 6]])
paths_2nd_2_2i = np.array([[3, 4]])
paths_2nd_2_2 = combine_paths(paths_2nd_2_2g, paths_2nd_2_2i)

points_2nd_2_3 = np.array([[0, 1], [0, 3], [1, 3], [2, 1], [3, 1], [3, 3]])
paths_2nd_2_3g = np.array([[1, 4], [2, 3], [3, 6], [4, 5]])
paths_2nd_2_3i = np.array([[3, 4]])
paths_2nd_2_3 = combine_paths(paths_2nd_2_3g, paths_2nd_2_3i)

points_2nd_2_4 = np.array([[0, 1], [0, 3], [1, 1], [2, 3], [3, 1], [3, 3]])
paths_2nd_2_4g = np.array([[1, 4], [2, 3], [3, 5], [4, 6]])
paths_2nd_2_4i = np.array([[3, 4]])
paths_2nd_2_4 = combine_paths(paths_2nd_2_4g, paths_2nd_2_4i)

points_2nd_2_5 = np.array([[0, 1], [0, 3], [1, 3], [2, 1], [3, 1], [3, 3]])
paths_2nd_2_5g = np.array([[1, 3], [2, 4], [3, 6], [4, 5]])
paths_2nd_2_5i = np.array([[3, 4]])
paths_2nd_2_5 = combine_paths(paths_2nd_2_5g, paths_2nd_2_5i)

points_2nd_3_1 = np.array([[0, 1], [2, 3], [3, 4], [4, 1], [4, 3], [4, 5]])
paths_2nd_3_1g = np.array([[1, 2], [2, 4], [3, 5], [3, 6]])
paths_2nd_3_1i = np.array([[2, 3]])
paths_2nd_3_1 = combine_paths(paths_2nd_3_1g, paths_2nd_3_1i)

points_2nd_3_2 = np.array([[0, 1], [2, 3], [3, 2], [4, 1], [4, 3], [4, 5]])
paths_2nd_3_2g = np.array([[1, 2], [3, 4], [3, 5], [2, 6]])
paths_2nd_3_2i = np.array([[2, 3]])
paths_2nd_3_2 = combine_paths(paths_2nd_3_2g, paths_2nd_3_2i)

points_2nd_3_3 = np.array([[0, 1], [2, 3], [3, 4], [4, 1], [4, 3], [4, 5]])
paths_2nd_3_3g = np.array([[1, 2], [3, 4], [2, 5], [3, 6]])
paths_2nd_3_3i = np.array([[2, 3]])
paths_2nd_3_3 = combine_paths(paths_2nd_3_3g, paths_2nd_3_3i)

can_points_2nd = np.empty((11, max(len(points_2nd_1_1), len(points_2nd_1_2), len(points_2nd_1_3)), 2))
can_points_2nd[0] = points_2nd_1_1
can_points_2nd[1] = points_2nd_1_2
can_points_2nd[2] = points_2nd_1_3
can_points_2nd[3] = points_2nd_2_1
can_points_2nd[4] = points_2nd_2_2
can_points_2nd[5] = points_2nd_2_3
can_points_2nd[6] = points_2nd_2_4
can_points_2nd[7] = points_2nd_2_5
can_points_2nd[8] = points_2nd_3_1
can_points_2nd[9] = points_2nd_3_2
can_points_2nd[10] = points_2nd_3_3

can_paths_2nd = np.empty((11, max(len(paths_2nd_1_1), len(paths_2nd_1_2), len(paths_2nd_1_3)), max(len(paths_2nd_1_1[0]), len(paths_2nd_1_2[0]), len(paths_2nd_1_3[0])), 2), dtype=int)
can_paths_2nd[0] = paths_2nd_1_1
can_paths_2nd[1] = paths_2nd_1_2
can_paths_2nd[2] = paths_2nd_1_3
can_paths_2nd[3] = paths_2nd_2_1
can_paths_2nd[4] = paths_2nd_2_2
can_paths_2nd[5] = paths_2nd_2_3
can_paths_2nd[6] = paths_2nd_2_4
can_paths_2nd[7] = paths_2nd_2_5
can_paths_2nd[8] = paths_2nd_3_1
can_paths_2nd[9] = paths_2nd_3_2
can_paths_2nd[10] = paths_2nd_3_3

can_number_2nd = np.array([[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]])

can_points = [can_points_1st, can_points_2nd]
can_paths = [can_paths_1st, can_paths_2nd]
can_count = [can_number_1st, can_number_2nd]

def return_counterterm_diagrams(order):
    """
    Generate counterterm diagrams up to the specified perturbative order,
    using NumPy arrays for better performance and clarity.
    """
    all_points = []
    all_paths = []

    for i in range(2, order + 1):
        points_orders = []
        paths_orders = []

        if i > 3:
            j = i - 4
            if j >= 0 and j < len(all_points):  # safeguard
                points_orders.extend(all_points[j])
                paths_orders.extend(all_paths[j])

        for j in range(1, i):
            k = i - j

            # Create points:
            # Outgoing: [1, 1], ..., [1, j]
            out_pts = np.stack([np.full(j, 1), np.arange(1, j + 1)], axis=1)

            # Counterterm: [2, 1]
            ct_pt = np.array([[2, 1]])

            # Incoming: [3, 1], ..., [3, k]
            in_pts = np.stack([np.full(k, 3), np.arange(1, k + 1)], axis=1)

            points = np.vstack([out_pts, ct_pt, in_pts])
            points_orders.append(points)

            # Create paths:
            # Outgoing: [1, j+1], ..., [j, j+1]
            out_paths = np.stack([np.arange(1, j + 1), np.full(j, j + 1)], axis=1)

            # Counterterm: [j+1, j+1]
            ct_path = np.array([[j + 1, j + 1]])

            # Incoming: [j+1, j+2], ..., [j+1, j+k+1]
            in_paths = np.stack([np.full(k, j + 1), np.arange(j + 2, j + k + 2)], axis=1)

            paths = np.vstack([out_paths, ct_path, in_paths])
            paths_orders.append(paths)

        all_points.append(points_orders)
        all_paths.append(paths_orders)

    return all_points, all_paths

counter_points, counter_paths = return_counterterm_diagrams(7)
for i in range(len(counter_points)):
    for j in range(len(counter_points[i])):
        print(f"Order {i + 2}, Diagram {j + 1}:")
        print("Points:", counter_points[i][j])
        print("Paths:",  np.array([counter_paths[i][j],np.zeros_like(counter_paths[i][j])]))
        print()

Order 2, Diagram 1:
Points: [[1 1]
 [2 1]
 [3 1]]
Paths: [[[1 2]
  [2 2]
  [2 3]]

 [[0 0]
  [0 0]
  [0 0]]]

Order 3, Diagram 1:
Points: [[1 1]
 [2 1]
 [3 1]
 [3 2]]
Paths: [[[1 2]
  [2 2]
  [2 3]
  [2 4]]

 [[0 0]
  [0 0]
  [0 0]
  [0 0]]]

Order 3, Diagram 2:
Points: [[1 1]
 [1 2]
 [2 1]
 [3 1]]
Paths: [[[1 3]
  [2 3]
  [3 3]
  [3 4]]

 [[0 0]
  [0 0]
  [0 0]
  [0 0]]]

Order 4, Diagram 1:
Points: [[1 1]
 [2 1]
 [3 1]]
Paths: [[[1 2]
  [2 2]
  [2 3]]

 [[0 0]
  [0 0]
  [0 0]]]

Order 4, Diagram 2:
Points: [[1 1]
 [2 1]
 [3 1]
 [3 2]
 [3 3]]
Paths: [[[1 2]
  [2 2]
  [2 3]
  [2 4]
  [2 5]]

 [[0 0]
  [0 0]
  [0 0]
  [0 0]
  [0 0]]]

Order 4, Diagram 3:
Points: [[1 1]
 [1 2]
 [2 1]
 [3 1]
 [3 2]]
Paths: [[[1 3]
  [2 3]
  [3 3]
  [3 4]
  [3 5]]

 [[0 0]
  [0 0]
  [0 0]
  [0 0]
  [0 0]]]

Order 4, Diagram 4:
Points: [[1 1]
 [1 2]
 [1 3]
 [2 1]
 [3 1]]
Paths: [[[1 4]
  [2 4]
  [3 4]
  [4 4]
  [4 5]]

 [[0 0]
  [0 0]
  [0 0]
  [0 0]
  [0 0]]]

Order 5, Diagram 1:
Points: [[1 1]
 [2 1]
 [3 

In [13]:
with open("counter_diagrams.py", "w") as f:
    f.write("counter_points = " + str(counter_points) + "\n")
    f.write("counter_paths = " + str(counter_paths) + "\n")